In [104]:
import os
import re
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.impute import KNNImputer
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [44]:
x_train = pd.read_csv('X_train.csv')
macro_train = pd.read_csv('macro_train.csv')
x_test = pd.read_csv('X_forward_looking.csv')
macro_test = pd.read_csv('macro_forward_looking.csv')
target_train = pd.read_csv('targets_train.csv')
sub = pd.read_csv('sample_submission.csv')

macro_train = macro_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
macro_test = macro_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [88]:
macro_train = pd.read_csv('macro_train.csv')
macro_test = pd.read_csv('macro_forward_looking.csv')
macro_train = macro_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
macro_test = macro_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [45]:
x_train = x_train.drop(columns=['companyId','lastUpdatedAnnumEndDate','lastUpdatedQuarterEndDate'])
macro_train = macro_train.drop(columns=['companyId'])
x_test = x_test.drop(columns=['companyId','lastUpdatedAnnumEndDate','lastUpdatedQuarterEndDate'])
macro_test = macro_test.drop(columns=['companyId'])
target_train = target_train.drop(columns=['companyId'])


In [89]:
macro_train = macro_train.drop(columns=['companyId'])
macro_test = macro_test.drop(columns=['companyId'])


In [46]:
def encode_categorical_features(dataset):
    # Define the features to encode
    categorical_features = ['metadata_0', 'industry', 'sector']
    
    # Initialize the LabelEncoder
    label_encoder = LabelEncoder()
    
    # Loop through each categorical feature and apply LabelEncoder
    for feature in categorical_features:
        if feature in dataset.columns:
            dataset[feature] = label_encoder.fit_transform(dataset[feature])
    
    return dataset

# Example usage:
# encoded_dataset = encode_categorical_features(your_dataset)

In [47]:
x_train = encode_categorical_features(x_train)
x_test = encode_categorical_features(x_test)

missing value imputation ? 

In [6]:

def apply_variance_thresholding(x_train, threshold=0.01):
    """
    Apply Variance Thresholding to the x_train dataset to discard features with low variance.

    Parameters:
    - x_train (DataFrame): The input dataset with features.
    - threshold (float): The variance threshold. Features with variance below this threshold will be removed.

    Returns:
    - x_train_reduced (DataFrame): The dataset with low-variance features removed.
    - removed_features (list): List of features that were removed.
    """
    # Initialize the VarianceThreshold with the given threshold
    selector = VarianceThreshold(threshold=threshold)

    # Fit and transform the data
    x_train_reduced = selector.fit_transform(x_train)

    # Get the mask of retained features
    retained_features_mask = selector.get_support()

    # Get the list of removed features
    removed_features = [column for column, retain in zip(x_train.columns, retained_features_mask) if not retain]

    # Convert the reduced array back to a DataFrame, keeping the retained feature names
    x_train_reduced = pd.DataFrame(x_train_reduced, columns=x_train.columns[retained_features_mask])

    return x_train_reduced, removed_features

# Example usage
x_train_reduced, removed_features = apply_variance_thresholding(x_train, threshold=0.01)

print("Reduced x_train shape:", x_train_reduced.shape)
print("Removed features:", removed_features)

In [48]:
def apply_variance_thresholding(x_train, threshold=0.00005):
    """
    Apply Variance Thresholding to the x_train dataset to discard features with low variance.

    Parameters:
    - x_train (DataFrame): The input dataset with features.
    - threshold (float): The variance threshold. Features with variance below this threshold will be removed.

    Returns:
    - x_train_reduced (DataFrame): The dataset with low-variance features removed.
    - removed_features (list): List of features that were removed.
    """
    # Initialize the VarianceThreshold with the given threshold
    selector = VarianceThreshold(threshold=threshold)

    # Fit and transform the data
    x_train_reduced = selector.fit_transform(x_train)

    # Get the mask of retained features
    retained_features_mask = selector.get_support()

    # Get the list of removed features
    removed_features = [column for column, retain in zip(x_train.columns, retained_features_mask) if not retain]

    # Convert the reduced array back to a DataFrame, keeping the retained feature names
    x_train_reduced = pd.DataFrame(x_train_reduced, columns=x_train.columns[retained_features_mask])

    return x_train_reduced, removed_features


In [49]:
x_train, removed_features = apply_variance_thresholding(x_train, threshold=0.00005)

print("Reduced x_train shape:", x_train.shape)
print("Removed features:", removed_features)

Reduced x_train shape: (19733, 1049)
Removed features: ['metadata_8', 'city', 'Q_4_feature_95', 'Q_4_feature_11', 'Q_4_feature_5', 'Q_4_feature_26', 'Q_4_feature_2', 'Q_4_feature_129', 'Q_4_feature_57', 'Q_4_feature_91', 'Q_4_feature_97', 'Q_4_feature_133', 'Q_4_feature_36', 'Q_4_feature_116', 'Q_3_feature_95', 'Q_3_feature_11', 'Q_3_feature_5', 'Q_3_feature_77', 'Q_3_feature_72', 'Q_3_feature_26', 'Q_3_feature_2', 'Q_3_feature_129', 'Q_3_feature_57', 'Q_3_feature_119', 'Q_3_feature_116', 'Q_2_feature_95', 'Q_2_feature_11', 'Q_2_feature_5', 'Q_2_feature_26', 'Q_2_feature_64', 'Q_2_feature_2', 'Q_2_feature_129', 'Q_2_feature_57', 'Q_1_feature_10', 'Q_1_feature_71', 'Q_1_feature_6', 'Q_1_feature_17', 'Q_1_feature_5', 'Q_1_feature_77', 'Q_1_feature_72', 'Q_1_feature_26', 'Q_1_feature_87', 'Q_1_feature_30', 'Q_1_feature_64', 'Q_1_feature_53', 'Q_1_feature_2', 'Q_1_feature_129', 'Q_1_feature_120', 'Q_1_feature_104', 'Q_1_feature_116', 'Y_3_feature_95', 'Y_3_feature_11', 'Y_3_feature_17', 'Y

C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\feature_selection\_variance_threshold.py:112: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)


In [50]:
x_train.head()

,metadata_0,industry,sector,metadata_1,metadata_2,metadata_3,metadata_4,metadata_5,metadata_6,metadata_7,...,Y_0_feature_22,Y_0_feature_69,Y_0_feature_108,Y_0_feature_123,Y_0_feature_80,Y_0_feature_13,Y_0_feature_20,Y_0_feature_35,Y_0_feature_45,Y_0_feature_16
0,69.0,8.0,5.0,0.003054,0.490076,0.003586,0.000486,0.000305,0.102313,0.000000,...,8.460000e-09,0.001202,2.487100e-04,0.000063,0.000065,0.999062,0.000010,0.001340,0.000270,0.015111
1,10.0,51.0,2.0,NaN,0.489839,0.000016,0.000048,0.000305,0.102182,0.000796,...,6.550000e-06,0.001189,5.390000e-05,0.000132,0.000206,0.998454,0.000284,0.001227,0.000258,0.015089
2,63.0,39.0,7.0,0.007253,0.486295,0.000016,0.000034,0.000305,0.102182,0.000000,...,6.900000e-07,0.001178,8.220000e-06,0.000092,0.000012,0.999051,0.000010,0.001212,0.000237,0.015103
3,34.0,20.0,7.0,0.000382,NaN,0.000001,0.000015,0.000305,NaN,NaN,...,8.460000e-09,0.001176,8.870000e-07,0.000063,0.000007,0.999062,0.000010,0.001212,0.000235,0.015090
4,69.0,17.0,6.0,NaN,0.485822,0.000802,0.000025,0.000305,0.101921,NaN,...,1.350000e-07,0.001189,8.510000e-05,0.000063,0.000006,0.999062,0.000010,0.001432,0.000245,0.015023


In [51]:
def remove_extra_features(x_train, x_test):
    """
    Remove features from x_test that are not present in x_train.

    Parameters:
    - x_train (DataFrame): The training dataset after feature selection.
    - x_test (DataFrame): The test dataset to be aligned with x_train.

    Returns:
    - x_test_reduced (DataFrame): The x_test dataset with extra features removed.
    - removed_features (list): List of features that were removed from x_test.
    """
    # Get the list of features in x_train
    train_features = set(x_train.columns)

    # Get the list of features in x_test
    test_features = set(x_test.columns)

    # Identify features in x_test that are not in x_train
    extra_features = test_features - train_features

    # Remove the extra features from x_test
    x_test_reduced = x_test.drop(columns=list(extra_features))

    return x_test_reduced, list(extra_features)

In [52]:
# Example usage
x_test, removed_features = remove_extra_features(x_train, x_test)

print("Reduced x_test shape:", x_test.shape)
print("Removed features from x_test:", removed_features)

Reduced x_test shape: (24119, 1049)
Removed features from x_test: ['Y_2_feature_53', 'Y_1_feature_31', 'Q_3_feature_2', 'Q_3_feature_26', 'Q_1_feature_72', 'Y_3_feature_2', 'Y_1_feature_70', 'Q_4_feature_91', 'Q_2_feature_11', 'Y_1_feature_86', 'Y_1_feature_129', 'Y_1_feature_98', 'Y_3_feature_17', 'Q_3_feature_5', 'Y_3_feature_57', 'Y_3_feature_94', 'city', 'Q_1_feature_6', 'Q_4_feature_2', 'Y_2_feature_64', 'Q_1_feature_26', 'Y_3_feature_53', 'Y_0_feature_116', 'Y_2_feature_75', 'Y_2_feature_81', 'Q_4_feature_26', 'Y_0_feature_104', 'Q_3_feature_129', 'Y_2_feature_74', 'Y_1_feature_2', 'Q_4_feature_133', 'Y_3_feature_75', 'Q_4_feature_116', 'Q_2_feature_57', 'Q_3_feature_116', 'Q_2_feature_129', 'Q_1_feature_30', 'Y_2_feature_26', 'Y_3_feature_95', 'Q_4_feature_129', 'Y_1_feature_94', 'Y_3_feature_74', 'Q_1_feature_53', 'Y_0_feature_2', 'Y_3_feature_81', 'Q_1_feature_64', 'Q_4_feature_97', 'Q_3_feature_119', 'Q_4_feature_36', 'Y_1_feature_116', 'Y_0_feature_64', 'Q_1_feature_71', 'Y_

In [90]:
macro_train, removed_features = apply_variance_thresholding(macro_train, threshold=0.00005)

print("Reduced x_train shape:", macro_train.shape)
print("Removed features:", removed_features)

Reduced x_train shape: (19733, 1099)
Removed features: ['MonetaryServicesIndexALLAssetsalternative_Q_4', 'OtherSeparationsTotalPrivate_Q_4', 'FederalGovernmentCurrentExpenditures_Q_4', 'FederalGovernmentCurrentReceipts_Q_4', 'HarmonizedIndexofConsumerPricesOverallIndexExcludingAlcoholandTobaccoforEuroArea18countries_Q_4', 'MonetaryServicesIndexALLAssetsalternative_Q_3', 'FederalGovernmentCurrentExpenditures_Q_3', 'FederalGovernmentCurrentReceipts_Q_3', 'HarmonizedIndexofConsumerPricesOverallIndexExcludingAlcoholandTobaccoforEuroArea18countries_Q_3', 'EffectiveFederalFundsRate_Q_2', 'MonetaryServicesIndexALLAssetsalternative_Q_2', 'FederalGovernmentCurrentExpenditures_Q_2', 'FederalGovernmentCurrentReceipts_Q_2', 'HarmonizedIndexofConsumerPricesOverallIndexExcludingAlcoholandTobaccoforEuroArea18countries_Q_2', 'EffectiveFederalFundsRate_Q_1', 'MonetaryServicesIndexALLAssetsalternative_Q_1', 'FederalGovernmentCurrentExpenditures_Q_1', 'FederalGovernmentCurrentReceipts_Q_1', 'HarmonizedIn

In [91]:
# Example usage
macro_test, removed_features = remove_extra_features(macro_train, macro_test)

print("Reduced x_test shape:", macro_test.shape)
print("Removed features from x_test:", removed_features)

Reduced x_test shape: (24119, 1099)
Removed features from x_test: ['DelinquencyRateonLeaseFinancingReceivablesAllCommercialBanks_Q_0_avg_90_days', 'TotalConsumerCreditSecuritizedbyDepositoryInstitutions_Q_0_min_90_days', 'AssetQualityMeasuresDelinquenciesonAllLoansandLeasesCommercialandIndustrialAllCommercialBanks_Q_0_avg_365_days', 'VelocityofM2MoneyStock_Q_0_min_90_days', 'DelinquencyRateonAllLoansAllCommercialBanks_Q_0_dynamic_90_days', 'DelinquencyRateonLoansSecuredbyRealEstateAllCommercialBanks_Q_0_max_90_days', 'DelinquencyRateonConsumerLoansAllCommercialBanks_Q_0_dynamic_90_days', 'CBOENASDAQ100VolatilityIndex_Q_0_min_365_days', 'AssetQualityMeasuresDelinquenciesonAllLoansandLeasesSecuredbyRealEstateAllCommercialBanks_Q_0_min_365_days', 'AssetQualityMeasuresDelinquenciesonAllLoansandLeasesToConsumersCreditCardsAllCommercialBanks_Q_0_dynamic_180_days', 'DelinquencyRateonLeaseFinancingReceivablesAllCommercialBanks_Q_0_max_365_days', 'FederalGovernmentCurrentExpenditures_Q_0', 'Del

In [58]:
def apply_knn_imputation(df, n_neighbors=5, weights='uniform'):
    """
    Apply KNN imputation to the numeric columns of a DataFrame.

    Parameters:
    - df (DataFrame): The input dataset with missing values.
    - n_neighbors (int): Number of neighbors to use for KNN imputation.
    - weights (str): Weight function used in prediction ('uniform' or 'distance').

    Returns:
    - df_imputed (DataFrame): The dataset with missing values imputed.
    """
    # Initialize KNNImputer with given parameters
    imputer = KNNImputer(n_neighbors=n_neighbors, weights=weights)

    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    
    # Apply KNN Imputer on the numeric columns
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
    
    return df


In [59]:
x_train = apply_knn_imputation(x_train)

In [62]:
x_test = apply_knn_imputation(x_test)


In [92]:
macro_train = apply_knn_imputation(macro_train)

In [93]:
macro_test = apply_knn_imputation(macro_test)

In [66]:
x_train.head()

,metadata_0,industry,sector,metadata_1,metadata_2,metadata_3,metadata_4,metadata_5,metadata_6,metadata_7,...,Y_0_feature_22,Y_0_feature_69,Y_0_feature_108,Y_0_feature_123,Y_0_feature_80,Y_0_feature_13,Y_0_feature_20,Y_0_feature_35,Y_0_feature_45,Y_0_feature_16
0,69.0,8.0,5.0,0.003054,0.490076,0.003586,0.000486,0.000305,0.102313,0.000000,...,8.460000e-09,0.001202,2.487100e-04,0.000063,0.000065,0.999062,0.000010,0.001340,0.000270,0.015111
1,10.0,51.0,2.0,0.004314,0.489839,0.000016,0.000048,0.000305,0.102182,0.000796,...,6.550000e-06,0.001189,5.390000e-05,0.000132,0.000206,0.998454,0.000284,0.001227,0.000258,0.015089
2,63.0,39.0,7.0,0.007253,0.486295,0.000016,0.000034,0.000305,0.102182,0.000000,...,6.900000e-07,0.001178,8.220000e-06,0.000092,0.000012,0.999051,0.000010,0.001212,0.000237,0.015103
3,34.0,20.0,7.0,0.000382,0.486200,0.000001,0.000015,0.000305,0.102261,0.000000,...,8.460000e-09,0.001176,8.870000e-07,0.000063,0.000007,0.999062,0.000010,0.001212,0.000235,0.015090
4,69.0,17.0,6.0,0.000611,0.485822,0.000802,0.000025,0.000305,0.101921,0.015292,...,1.350000e-07,0.001189,8.510000e-05,0.000063,0.000006,0.999062,0.000010,0.001432,0.000245,0.015023


In [67]:
x_test.head()

,metadata_0,industry,sector,metadata_1,metadata_2,metadata_3,metadata_4,metadata_5,metadata_6,metadata_7,...,Y_0_feature_22,Y_0_feature_69,Y_0_feature_108,Y_0_feature_123,Y_0_feature_80,Y_0_feature_13,Y_0_feature_20,Y_0_feature_35,Y_0_feature_45,Y_0_feature_16
0,10.0,87.0,0.0,0.001069,0.485066,0.000018,3.868582e-07,0.000305,0.102182,0.000087,...,8.803780e-09,0.001174,1.079832e-04,0.000063,1.565739e-07,0.999062,0.000010,0.001214,0.000232,0.015083
1,70.0,126.0,2.0,0.004123,0.481333,0.000065,5.784876e-07,0.000305,0.102208,0.000014,...,8.458420e-09,0.001174,8.020457e-05,0.000063,9.392309e-08,0.999062,0.000013,0.001212,0.000232,0.015090
2,10.0,87.0,0.0,0.000458,0.485255,0.000000,3.304119e-07,0.000305,0.102182,0.000087,...,8.458420e-09,0.001174,4.140925e-05,0.000063,2.649000e-07,0.999062,0.000010,0.001213,0.000233,0.015089
3,38.0,93.0,6.0,0.000000,0.486059,0.000010,1.014865e-03,0.000305,0.102835,0.000000,...,1.178768e-05,0.001277,4.689299e-07,0.000065,8.129469e-04,0.999062,0.000010,0.001226,0.000321,0.015404
4,10.0,88.0,0.0,0.011453,0.486106,0.000011,3.598743e-07,0.000305,0.102182,0.000043,...,8.458420e-09,0.001174,6.772559e-05,0.000063,6.856803e-08,0.999062,0.000010,0.001212,0.000232,0.015090


In [94]:
macro_train.head()

,EffectiveFederalFundsRate_Q_4,SecuredOvernightFinancingRate_Q_4,15YearFixedRateMortgageAverageintheUnitedStates_Q_4,30YearFixedRateMortgageAverageintheUnitedStates_Q_4,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan_Q_4,MarketYieldonUSTreasurySecuritiesat1MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat3MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat6MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat1YearConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat2YearConstantMaturityQuotedonanInvestmentBasis_Q_4,...,HiresTotalPrivate_Y_0,TotalSeparationsTotalPrivate_Y_0,QuitsTotalPrivate_Y_0,LayoffsandDischargesTotalPrivate_Y_0,OtherSeparationsTotalPrivate_Y_0,ContinuedClaimsInsuredUnemployment_Y_0,InitialClaims_Y_0,GDPNow_Y_0,BusinessExpectationsEmploymentGrowth_Y_0,BusinessTendencySurveysEmploymentEconomicActivityServicesTendencyforEuroArea19Countries_Y_0
0,0.666667,0.717949,0.403846,0.328571,0.0,0.336,0.216867,0.161765,0.0,0.047059,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725
1,0.666667,0.717949,0.403846,0.328571,0.0,0.336,0.216867,0.161765,0.0,0.047059,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725
2,0.666667,0.717949,0.403846,0.328571,0.0,0.336,0.216867,0.161765,0.0,0.047059,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725
3,0.666667,0.717949,0.403846,0.328571,0.0,0.336,0.216867,0.161765,0.0,0.047059,...,0.666667,0.400000,0.454545,0.000000,0.0,0.856031,0.421053,0.346534,1.000000,0.329670
4,0.666667,0.717949,0.403846,0.328571,0.0,0.336,0.216867,0.161765,0.0,0.047059,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725


In [95]:
macro_test.head()

,EffectiveFederalFundsRate_Q_4,SecuredOvernightFinancingRate_Q_4,15YearFixedRateMortgageAverageintheUnitedStates_Q_4,30YearFixedRateMortgageAverageintheUnitedStates_Q_4,FinanceRateonPersonalLoansatCommercialBanks24MonthLoan_Q_4,MarketYieldonUSTreasurySecuritiesat1MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat3MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat6MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat1YearConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat2YearConstantMaturityQuotedonanInvestmentBasis_Q_4,...,HiresTotalPrivate_Y_0,TotalSeparationsTotalPrivate_Y_0,QuitsTotalPrivate_Y_0,LayoffsandDischargesTotalPrivate_Y_0,OtherSeparationsTotalPrivate_Y_0,ContinuedClaimsInsuredUnemployment_Y_0,InitialClaims_Y_0,GDPNow_Y_0,BusinessExpectationsEmploymentGrowth_Y_0,BusinessTendencySurveysEmploymentEconomicActivityServicesTendencyforEuroArea19Countries_Y_0
0,0.32,0.307692,0.721154,0.8,0.0,0.528,0.253012,0.558824,0.597403,1.0,...,0.333333,0.066667,0.181818,0.000000,0.0,0.828794,0.210526,0.346534,0.846809,0.362637
1,0.32,0.307692,0.721154,0.8,0.0,0.528,0.253012,0.558824,0.597403,1.0,...,0.333333,0.066667,0.181818,0.000000,0.0,0.828794,0.210526,0.346534,0.846809,0.362637
2,0.32,0.307692,0.721154,0.8,0.0,0.528,0.253012,0.558824,0.597403,1.0,...,0.000000,0.266667,0.090909,0.428571,0.0,0.887160,0.315789,0.367949,0.406809,0.219780
3,0.32,0.307692,0.721154,0.8,0.0,0.528,0.253012,0.558824,0.597403,1.0,...,1.000000,0.600000,0.727273,0.142857,0.0,0.599222,1.000000,1.000000,0.356596,0.582418
4,0.32,0.307692,0.721154,0.8,0.0,0.528,0.253012,0.558824,0.597403,1.0,...,0.333333,0.066667,0.181818,0.000000,0.0,0.828794,0.210526,0.346534,0.846809,0.362637


In [70]:

def filter_strong_features(x_train, target_train, corr_threshold=0.5):
    """
    Filters features in x_train based on their correlation with target_train.

    Parameters:
    - x_train: pd.DataFrame with independent features
    - target_train: pd.DataFrame with multiple target variables
    - corr_threshold: float, correlation threshold to classify features as strong

    Returns:
    - pd.DataFrame with features having strong relationship with target_train
    """
    
    # Combine x_train and target_train into one DataFrame
    combined = pd.concat([x_train, target_train], axis=1)
    
    # Calculate the correlation matrix
    corr_matrix = combined.corr()
    
    # Extract correlations between features in x_train and each target variable
    feature_target_corr = corr_matrix.loc[x_train.columns, target_train.columns]
    
    # Initialize a list to keep track of strong features
    strong_features = []

    for feature in x_train.columns:
        # Calculate the maximum correlation with all target variables
        max_corr = feature_target_corr.loc[feature].abs().max()
        
        if max_corr >= corr_threshold:
            strong_features.append(feature)
    
    # Return x_train with only strong features
    return x_train[strong_features]

In [71]:
x_train = filter_strong_features(x_train, target_train, corr_threshold=0.5)
x_train.head()

,Q_4_feature_71,Q_4_feature_78,Q_4_feature_86,Q_4_feature_27,Q_4_feature_6,Q_4_feature_48,Q_4_feature_24,Q_4_feature_17,Q_4_feature_14,Q_4_feature_46,...,Y_0_feature_111,Y_0_feature_99,Y_0_feature_22,Y_0_feature_69,Y_0_feature_123,Y_0_feature_80,Y_0_feature_13,Y_0_feature_35,Y_0_feature_45,Y_0_feature_16
0,0.054146,0.000489,0.033458,0.018108,0.029896,0.091234,0.031992,0.966239,0.000220,0.008637,...,0.000140,7.440000e-10,8.460000e-09,0.001202,0.000063,0.000065,0.999062,0.001340,0.000270,0.015111
1,0.054040,0.000595,0.033235,0.018139,0.030261,0.091156,0.031769,0.966185,0.000218,0.008783,...,0.000023,2.001780e-04,6.550000e-06,0.001189,0.000132,0.000206,0.998454,0.001227,0.000258,0.015089
2,0.054040,0.000471,0.033391,0.018051,0.029352,0.091231,0.031926,0.966286,0.000222,0.008623,...,0.000004,2.160000e-05,6.900000e-07,0.001178,0.000092,0.000012,0.999051,0.001212,0.000237,0.015103
3,0.054040,0.000461,0.033383,0.018056,0.029352,0.091173,0.031917,0.966280,0.000220,0.008612,...,0.000002,1.020000e-05,8.460000e-09,0.001176,0.000063,0.000007,0.999062,0.001212,0.000235,0.015090
4,0.054268,0.000457,0.033293,0.018051,0.029510,0.091136,0.031827,0.966324,0.000218,0.008618,...,0.000002,7.440000e-10,1.350000e-07,0.001189,0.000063,0.000006,0.999062,0.001432,0.000245,0.015023


In [72]:
# Example usage
x_test, removed_features = remove_extra_features(x_train, x_test)

print("Reduced x_test shape:", x_test.shape)
print("Removed features from x_test:", removed_features)

Reduced x_test shape: (24119, 762)
Removed features from x_test: ['Q_1_feature_76', 'Q_1_feature_38', 'Q_4_feature_75', 'Q_4_feature_13', 'Y_3_feature_116', 'Y_1_feature_75', 'Y_2_feature_108', 'Y_1_feature_38', 'Y_2_feature_92', 'Q_3_feature_0', 'Q_4_feature_72', 'Y_3_feature_108', 'Q_1_feature_96', 'Q_4_feature_41', 'Y_1_feature_0', 'Q_3_feature_100', 'metadata_9', 'Y_3_feature_21', 'Q_1_feature_0', 'Q_1_feature_67', 'Y_2_feature_139', 'Q_2_feature_8', 'Y_0_feature_4', 'Y_2_feature_96', 'Q_3_feature_113', 'Q_2_feature_138', 'Y_1_feature_96', 'Y_1_feature_72', 'Q_4_feature_38', 'Y_2_feature_121', 'Q_2_feature_58', 'Y_0_feature_20', 'Q_2_feature_119', 'metadata_2', 'Y_2_feature_0', 'Q_1_feature_75', 'Q_1_feature_91', 'Y_2_feature_131', 'Y_3_feature_62', 'Y_1_feature_57', 'Q_4_feature_20', 'Q_4_feature_58', 'Y_1_feature_108', 'Y_3_feature_44', 'Q_4_feature_40', 'Y_2_feature_21', 'Q_4_feature_44', 'Y_3_feature_100', 'Y_3_feature_92', 'Y_3_feature_41', 'Y_0_feature_40', 'Y_0_feature_121',

In [98]:
def select_important_features_lgbm(macro_train, target_train, importance_threshold=0.1):
    """
    Selects important features from macro_train based on their importance in predicting target_train.
    
    Parameters:
    - macro_train: pd.DataFrame with independent features
    - target_train: pd.DataFrame with multiple target variables
    - importance_threshold: float, minimum importance score to consider a feature important
    
    Returns:
    - pd.DataFrame with only the important features
    """

    # Initialize an empty DataFrame to store feature importances
    feature_importances = pd.DataFrame(index=macro_train.columns)

    # Iterate over each target variable
    for target in target_train.columns:
        # Create and train the LightGBM model
        model = lgb.LGBMRegressor()
        model.fit(macro_train, target_train[target])

        # Store the feature importances
        feature_importances[target] = model.feature_importances_

    # Calculate the mean importance across all target variables
    feature_importances['mean_importance'] = feature_importances.mean(axis=1)

    # Filter out features with mean importance below the threshold
    important_features = feature_importances[feature_importances['mean_importance'] >= importance_threshold].index

    # Return the macro_train with only the important features
    return macro_train[important_features]

In [99]:
macro_train = select_important_features_lgbm(macro_train, target_train, importance_threshold=0.1)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6884
[LightGBM] [Info] Number of data points in the train set: 19733, number of used features: 1086
[LightGBM] [Info] Start training from score 0.000211
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.093247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6884
[LightGBM] [Info] Number of data points in the train set: 19733, 

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.106938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6884
[LightGBM] [Info] Number of data points in the train set: 19733, number of used features: 1086
[LightGBM] [Info] Start training from score 0.000198
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [100]:
macro_train.head()

,EffectiveFederalFundsRate_Q_4,SecuredOvernightFinancingRate_Q_4,15YearFixedRateMortgageAverageintheUnitedStates_Q_4,MarketYieldonUSTreasurySecuritiesat1MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat3MonthConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat1YearConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat2YearConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat5YearConstantMaturityQuotedonanInvestmentBasis_Q_4,MarketYieldonUSTreasurySecuritiesat30YearConstantMaturityQuotedonanInvestmentBasis_Q_4,TotalBorrowingsfromtheFederalReserve_Q_4,...,HiresTotalPrivate_Y_0,TotalSeparationsTotalPrivate_Y_0,QuitsTotalPrivate_Y_0,LayoffsandDischargesTotalPrivate_Y_0,OtherSeparationsTotalPrivate_Y_0,ContinuedClaimsInsuredUnemployment_Y_0,InitialClaims_Y_0,GDPNow_Y_0,BusinessExpectationsEmploymentGrowth_Y_0,BusinessTendencySurveysEmploymentEconomicActivityServicesTendencyforEuroArea19Countries_Y_0
0,0.666667,0.717949,0.403846,0.336,0.216867,0.0,0.047059,0.152778,0.411765,1.0,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725
1,0.666667,0.717949,0.403846,0.336,0.216867,0.0,0.047059,0.152778,0.411765,1.0,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725
2,0.666667,0.717949,0.403846,0.336,0.216867,0.0,0.047059,0.152778,0.411765,1.0,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725
3,0.666667,0.717949,0.403846,0.336,0.216867,0.0,0.047059,0.152778,0.411765,1.0,...,0.666667,0.400000,0.454545,0.000000,0.0,0.856031,0.421053,0.346534,1.000000,0.329670
4,0.666667,0.717949,0.403846,0.336,0.216867,0.0,0.047059,0.152778,0.411765,1.0,...,0.500000,0.666667,0.363636,0.714286,1.0,0.657588,0.000000,0.367949,0.274043,0.274725


In [101]:
# Example usage
macro_test, removed_features = remove_extra_features(macro_train, macro_test)

print("Reduced x_test shape:", macro_train .shape)
print("Removed features from x_test:", removed_features)

Reduced x_test shape: (19733, 233)
Removed features from x_test: ['BusinessTendencySurveysEmploymentEconomicActivityServicesTendencyforEuroArea19Countries_Q_0_avg_180_days', 'BusinessExpectationsEmploymentGrowth_Q_3', 'DelinquencyRateonCommercialRealEstateLoansExcludingFarmlandBookedinDomesticOfficesAllCommercialBanks_Y_2', 'ConsumerPriceIndexforAllUrbanConsumersAllItemsinUSCityAverage_Q_0_avg_90_days', 'AverageHourlyEarningsofAllEmployeesTotalPrivate_Q_0_max_365_days', 'DowJonesTransportationAverage_Q_0', 'ChicagoFedNationalActivityIndexPersonalConsumptionandHousing_Q_0_avg_180_days', 'MarketYieldonUSTreasurySecuritiesat30YearConstantMaturityQuotedonanInvestmentBasis_Q_0_avg_180_days', 'OtherSeparationsTotalPrivate_Q_2', 'QuitsTotalPrivate_Q_1', '30YearFixedRateMortgageAverageintheUnitedStates_Q_0_min_90_days', 'AssetQualityMeasuresDelinquenciesonAllLoansandLeasesToConsumersCreditCardsAllCommercialBanks_Y_1', 'DelinquencyRateonConsumerLoansAllCommercialBanks_Y_1', 'TotalSeparationsTot

In [105]:
X_train_combined = pd.concat([x_train, macro_train], axis=1)
# Initialize the RandomForest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the concatenated training data
rf_model.fit(X_train_combined, target_train)

RandomForestRegressor(random_state=42)

In [107]:
# Concatenate the testing datasets
X_test_combined = pd.concat([x_test, macro_test], axis=1)

# Make predictions on the concatenated test data
predictions = rf_model.predict(X_test_combined)

# Assuming your predictions array has the same number of columns as target variables
# Loop over each target and assign predictions to the corresponding column in sub
for i, col in enumerate(sub.columns[1:]):  # Skip the first column if it's an ID column
    sub[col] = predictions[:, i]

# Save the final submission file
sub.to_csv('submission.csv', index=False)

print("Predictions have been saved to submission.csv")

Predictions have been saved to submission.csv
